<a href="https://colab.research.google.com/github/milanochuang/518job_parser/blob/master/job_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import datetime
import json
import ast

In [2]:
url = "https://www.518.com.tw/job-index.html"
url_2 = "https://www.518.com.tw/job-index-P-{}.html?ai=0&ai=0"

In [3]:
def jobURL_grabber(URL, jobUrlLIST):
  r = requests.get(url)
  jobpage = BeautifulSoup(r.text, "html.parser")
  for i in jobpage.find_all('a', href=True):
    if 'https://www.518.com.tw/job' in i['href']:
      if 'index' not in i['href']:
        if i['href'] in jobUrlLIST:
          continue
        else:
          jobUrlLIST.append(i['href'])
  return jobUrlLIST

In [4]:
pageIndex = 1
jobUrlLIST = list()
print("正在爬取第{}頁".format(pageIndex))
jobURL_grabber(url, jobUrlLIST)
for i in range(2, 3): #正式抓取應為151
    pageIndex += 1
    print("正在爬取第{}頁".format(pageIndex))
    jobURL_grabber(url_2.format(i), jobUrlLIST)
with open("518JobUrlLIST_{}.json".format(datetime.date.today()), "w", encoding="utf-8") as f:
        json.dump(jobUrlLIST, f, ensure_ascii=False)

正在爬取第1頁
正在爬取第2頁


In [5]:
def data_cleaner(string):
  jobSTR = [x.replace("\r","") for x in string]
  joblist = [x.replace("\n","") for x in jobSTR]
  return joblist

In [6]:
def jobLIST_grabber(jobUrl):
  resultDICT = {
        "jobTitle": "",
        "company": "",
        "salary":[''],
        "geoInfo":{"road":"",
                   "address": ""
                   },
        "majoring":[''],
        "tool":[''],
        "skills":[''],
        "certificate":[''],
        "experience":[''],
        "license":''
    }
  r = requests.get(jobUrl)
  jobpage = BeautifulSoup(r.text, "html.parser")
  title = data_cleaner(jobpage.find(class_ = "job-title"))[0]
  resultDICT['jobTitle'] = title
  salary = jobpage.find(class_ = 'jobItem').find("li").find("span").text[1:]
  resultDICT['salary'] = salary
  company = jobpage.select("a span")[1].text
  resultDICT['company'] = company
  address = jobpage.find(class_ = 'jobItem').find_all("li")[1].find("span").text[1:]
  resultDICT['geoInfo']['address'] = address
  condition = jobpage.find(class_ = 'condition').find_all('li')
  index = 1
  for i in condition:
    if i.find("span") != None:
      # print("第{}個".format(index), i.text[0:4], i.find("span").text)
      index+=1
      if '工作經驗' in i.text:
        if i.text[0:4] == "工作經驗":
          experience = i.find("span").text[1:]
          resultDICT['experience'] = experience
      else:
        experience = ''
      if '科系限制' in i.text:
        if i.text[0:4] == "科系限制":
          majoring = i.find("span").text
          resultDICT['majoring'] = majoring
      else:
        majoring = ''
      if '語文條件' in i.text:
        if i.text[0:4] == "語文條件":
          language = i.find("span").text[1:]
          language = language.replace("\xa0", " ")
          resultDICT['language'] = language
      else:
        language = ''
      if '擅長工具' in i.text:
        if i.text[0:4] == "擅長工具":
          tool = i.find("span").text[1:]
          resultDICT['tool'] = tool
      else:
        tool = ''
      if '工作技能' in i.text:
        if i.text[0:4] == "工作技能":
          if '駕照' in i.find("span").text:
            skills = i.find("span").text[1:3]
            resultDICT['skills'] = skills
          else:
            skills = i.find("span").text[1:]
            resultDICT['skills'] = skills
      else:
        skills = ''
      if '所需證照' in i.text:
        if i.text[0:4] == "所需證照":
          certificate = i.find("span").text
          resultDICT['certificate'] = certificate
      else:
        certificate = ''
      if '需備駕照' in i.text:
        if i.text[0:4] == "需備駕照":
          license = i.find("span").text
          resultDICT['license'] = license
      else:
        license = ''
  return resultDICT

In [7]:
resultLIST = list()
for jobUrl in jobUrlLIST:
  if jobUrlLIST.index(jobUrl) == int(len(jobUrlLIST)*0.25):
    print("已爬取25%的網頁")
  if jobUrlLIST.index(jobUrl) == int(len(jobUrlLIST)*0.5):
    print("已爬取50%的網頁")
  if jobUrlLIST.index(jobUrl) == int(len(jobUrlLIST)*0.75):
    print("已爬取75%的網頁")
  resultDICT = jobLIST_grabber(jobUrl)
  resultLIST.append(resultDICT)
print("爬取完畢")

已爬取25%的網頁
已爬取50%的網頁
已爬取75%的網頁
爬取完畢


In [8]:
for i in resultLIST:
  print(i)

{'jobTitle': '電訪人員', 'company': '合潤行銷有限公司', 'salary': '月薪 24,000 元以上', 'geoInfo': {'road': '', 'address': '高雄市苓雅區光華一路206號11樓之10'}, 'majoring': [''], 'tool': '不拘', 'skills': '不拘', 'certificate': [''], 'experience': '不拘', 'license': '', 'language': '中文 聽:精通 / 說:精通 / 讀:精通 / 寫:精通方言條件 台語 : 略懂 '}
{'jobTitle': '銷售助理', 'company': '錦卿企業有限公司', 'salary': '時薪 160 元以上', 'geoInfo': {'road': '', 'address': '新北市板橋區南雅南路二段104號1樓'}, 'majoring': [''], 'tool': '不拘', 'skills': '不拘', 'certificate': [''], 'experience': '不拘', 'license': '', 'language': '不拘'}
{'jobTitle': '水電師傅', 'company': '東岳修繕工程行', 'salary': '月薪 60,000 至 80,000 元', 'geoInfo': {'road': '', 'address': '新北市土城區延和路105巷7弄13號'}, 'majoring': [''], 'tool': '不拘', 'skills': '不拘', 'certificate': [''], 'experience': '不拘', 'license': '普通重型機車、普通小型車', 'language': '不拘'}
{'jobTitle': '店長', 'company': '美簾工坊', 'salary': '月薪 35,000 至 40,000 元', 'geoInfo': {'road': '', 'address': '台南市安南區安吉路一段202號'}, 'majoring': [''], 'tool': 'Excel、Word', 'skills': '工作', 'certifi

In [9]:
with open("518JobLIST_{}.json".format(datetime.date.today()), "w", encoding="utf-8") as f:
        json.dump(resultLIST, f, ensure_ascii=False)